# Введение в анализ данных
## НИУ ВШЭ, 2019-2020 учебный год

### Домашнее задание №3

Задание выполнил: Ковалев Владислав

### Общая информация
Дата выдачи: 8.04.2020

Дедлайн: 26.04.2020

### О задании

В этом домашнем задании вы будете работать с линейной классификацией, попрактикуетесь на реальной задаче классификации текстов.

Для решения этого домашнего задания намного удобнее будет использовать Colab, так как данных много.

### Оценивание и штрафы

За сдачу задания позже срока на итоговую оценку за задание накладывается штраф в размере 1 балл в день, но получить отрицательную оценку нельзя.

__Внимание!__ Домашнее задание выполняется самостоятельно. «Похожие» решения считаются плагиатом и все задействованные студенты (в том числе те, у кого списали) не могут получить за него больше 0 баллов.

### Формат сдачи
Загрузка файлов с решениями происходит в системе Anytask.

### Подготовка данных

In [1]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
%matplotlib inline

from PIL import Image
import requests
from io import BytesIO

import copy
from sklearn.preprocessing import normalize
from sklearn.model_selection import train_test_split
from nltk.tokenize import WordPunctTokenizer
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score
from tqdm import tqdm

In [2]:
# Датасет можно скачать здесь

!wget https://www.dropbox.com/s/tg55q9mrziroyrs/train_subset.csv

"wget" ­Ґ пў«пҐвбп ў­гваҐ­­Ґ© Ё«Ё ў­Ґи­Ґ©
Є®¬ ­¤®©, ЁбЇ®«­пҐ¬®© Їа®Ја ¬¬®© Ё«Ё Ї ЄҐв­л¬ д ©«®¬.


### Данные

Мы имеем дело с данными с торговой платформы Avito.
Для каждого товара представлены следующие параметры:
 - title
 - description
 - Category_name
 - Category

Имеется информация об объектах 50 классов.
Задача: по новым объектам (title, description) предсказать Category.
(Очевидно, что параметр Category_name для предсказания классов использовать нельзя)

In [3]:
data = pd.read_csv("train_subset.csv", index_col='id')

data.head()

,title,description,Category_name,Category
id,,,,
382220,Прихожая,В хорошем состоянии. Торг,Мебель и интерьер,20
397529,Кордиант 215/55/16 Летние,Кордиант 215/55/16 Летние/\n /\nАртикул: 1737l...,Запчасти и аксессуары,10
584569,Стол,"Стол, 2 рабочих места . Стол серого цвета, в д...",Мебель и интерьер,20
2513100,Комбинезон,Размер-42/44,"Одежда, обувь, аксессуары",27
1091886,Ветровка,На 2 года,Детская одежда и обувь,29


In [4]:
data.shape

(30000, 4)

In [5]:
X = data[['title', 'description']].to_numpy()
y = data['Category'].to_numpy()

del data

Сразу разделим выборку на train и test.
Никакие данные из test для обучения использовать нельзя!

In [6]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [7]:
X_train[:5]

array([['Сапоги 46 размер новые', 'Сапоги 46 размер новые'],
       ['Светильники потолочный swarovski',
        'светильники потолочные swarovski 6 штук , цена за штуку. В эксплуатации 2 года , продаются в связи со сменой интерьера в квартире'],
       ['iPhone 7 plus 128GB Red красный в наличии',
        '\xa0/\n/\n Данная цена только для подписчиков Instagram: iQmac/\n/\n Новый красный айфон 7 Plus в наличии это элегантный и мощный смартфон, который готов в полной мере раскрыть новые возможности iOS 10. Аппарат с 4-ядерным процессором А10 и 3 ГБ ОЗУ с легкостью решает самые ресурсоемкие задачи, позволяя наслаждаться быстродействием «тяжелых» приложений и игр на 5,5-дюймовом дисплее. Аппарат получил экран, как у iPad Pro, так что картинка теперь соответствует кинематографическому стандарту.'],
       ['Пион Ирис Ромашка рассада',
        'Пион куст 500 р ( более 10 шт)/\nСаженец/ корень 100р/\nРастут у нас более 70 лет/\nРозовые, бордовые и белые/\nНа фото цветы 2018г/\nП. Зубчанинов

### Токенизация (1 балл)


Токенизация -- разбиение текста на мелкие части, которые можно обработать машинными методами.
Можно использовать разные алгоритмы токенизации.
Давайте пока остановимся на простом WordPunctTokenizer.


In [8]:
tokenizer = WordPunctTokenizer()


def preprocess(text: str) -> str:
    return ' '.join(tokenizer.tokenize(text.lower()))


text = 'Здраствуйте. Я, Кирилл. Хотел бы чтобы вы сделали игру, 3Д-экшон суть такова...'
print("before:", text,)
print("after:", preprocess(text),)

before: Здраствуйте. Я, Кирилл. Хотел бы чтобы вы сделали игру, 3Д-экшон суть такова...
after: здраствуйте . я , кирилл . хотел бы чтобы вы сделали игру , 3д - экшон суть такова ...


__Задание:__ Токенизируйте title и description в train и test

In [9]:
len_train = len(X_train)
len_test = len(X_test)
X_train_dt = [0]*len_train
X_test_dt = [0]*len_test

In [10]:
for i in range(len(X_train)):
    for j in range(len(X_train[i])):
        X_train[i][j] = preprocess(X_train[i][j])
for i in range(len(X_test)):
    for j in range(len(X_test[i])):
        X_test[i][j] = preprocess(X_test[i][j])
for i in tqdm(range(len_train)):
    X_train_dt[i] = X_train[i][0]+ ' ' + X_train[i][1] 
for i in tqdm(range(len_test)):
    X_test_dt[i] = X_test[i][0]+ ' ' + X_test[i][1] 

100%|██████████████████████████████████████████████████████████████████████████| 9000/9000 [00:00<00:00, 121595.18it/s]


In [11]:
X_train[:5]

array([['сапоги 46 размер новые', 'сапоги 46 размер новые'],
       ['светильники потолочный swarovski',
        'светильники потолочные swarovski 6 штук , цена за штуку . в эксплуатации 2 года , продаются в связи со сменой интерьера в квартире'],
       ['iphone 7 plus 128gb red красный в наличии',
        '/ / данная цена только для подписчиков instagram : iqmac / / новый красный айфон 7 plus в наличии это элегантный и мощный смартфон , который готов в полной мере раскрыть новые возможности ios 10 . аппарат с 4 - ядерным процессором а10 и 3 гб озу с легкостью решает самые ресурсоемкие задачи , позволяя наслаждаться быстродействием « тяжелых » приложений и игр на 5 , 5 - дюймовом дисплее . аппарат получил экран , как у ipad pro , так что картинка теперь соответствует кинематографическому стандарту .'],
       ['пион ирис ромашка рассада',
        'пион куст 500 р ( более 10 шт )/ саженец / корень 100р / растут у нас более 70 лет / розовые , бордовые и белые / на фото цветы 2018г / п .

In [12]:
assert X_train[10][1] == 'продам иж планета 3 , 76 год , ( стоит на старом учёте , документы утеряны ) на ходу , хорошее состояние , все интересующие вопросы по телефону ( с родной коляской на 3 тысячи дороже ) . торга не будет .'

### BOW (1.5 балла)

Один из традиционных подходов -- построение bag of words.

Метод состоит в следующем:

 - Составить словарь самых часто встречающихся слов в train data
 - Для каждого примера из train посчитать, сколько раз каждое слово из словаря в нём встречается


 В sklearn есть CountVectorizer, но в этом задании его использовать нельзя.

__Задание:__ Найдите k самых частых слов, отсортируйте их по убыванию частотности (k=10000)

In [13]:
bow_vocabulary = []
support_dict = {}
for i in X_train:
    for j in i:
        for k in list(j.split()):
            try:
                support_dict[k] += 1
            except:
                support_dict.update({k:1})
sup_list = list(support_dict.items())
sup_list.sort(key=lambda x: x[1], reverse=True)
for i in range(10000):
    bow_vocabulary.append(sup_list[i][0])

In [14]:
assert sorted(bow_vocabulary)[::200] == ['!', '12500', '270', '700', 'by', 'gh', 'michael', 'sonata', 'ø', 'аудиоподготовка', 'большим', 'веса', 'воспроизведения', 'габариты', 'гтд', 'джинсами', 'доступность', 'загрузки', 'зимней', 'использовался', 'квартала', 'коммуникации', 'кошки', 'лакированные', 'магазин', 'металл', 'мск', 'натуральным', 'носке', 'одному', 'отвечаем', 'пассат', 'плотно', 'покраску', 'постоянные', 'примеры', 'просьба', 'размещайте', 'репетитор', 'сантехник', 'сидения', 'современного', 'стала', 'схема', 'тон', 'удлиненная', 'фасад', 'цветами', 'шея', 'эту']

In [15]:
def text_to_bow(text: str) -> np.array:
    """
    Возвращает вектор, где для каждого слова из most_common
    указано количество его употреблений
    """ 
    global result
    result = [0]*10000
    for i in list(text.split()):
        if i in bow_vocabulary:
            result[int(bow_vocabulary.index(i))] = list(text.split()).count(i)
    return np.array(result)

In [16]:
assert np.allclose(np.where(text_to_bow("сдаётся уютный , тёплый гараж для стартапов в ml") != 0)[0],
                   np.array([   1,    4,   12,  565,  866, 1601, 2539, 4063])
)

In [17]:
def items_to_bow(items: np.array) -> np.array:
    """ Для каждого товара возвращает вектор его bow """
    # Давайте для начала попробуем строить bow только из description товара
    # assert ниже написан для bow из description
    matrix = [0]*len(items)
    for i in tqdm(range(len(items))):
          matrix[i] = text_to_bow(items[i][1])
    return np.array(matrix)

In [ ]:
assert np.allclose(np.where(items_to_bow([X_train[42]])[0] != 0),
                   np.array([   0, 1, 2, 5, 6, 7, 12, 27, 41, 49, 110,
                                189,  208,  221, 2032, 3052, 7179, 9568]),)

In [1]:
X_train_bow = items_to_bow(X_train)
X_test_bow = items_to_bow(X_test)

NameError: name 'items_to_bow' is not defined

### Логистическая регрессия и SVC (1 балл)


Теперь описание каждого товара представлено, как точка в многомерном пространстве.
Очень важно запомнить эту идею: дальше мы будем рассматривать разные способы перехода от текста к точке в пространстве.

Для BOW каждое измерение в пространстве -- какое-то слово.
Мы предполагаем, что текст описывается набором каких-то популярных слов, которые в нём встречаются, а близкие по смыслу тексты будут использовать одинаковые слова.

Обучите логистическую регрессию и SVC с базовыми параметрами.


In [2]:
bow_model = LogisticRegression(max_iter=100).fit(X_train_bow, y_train)
print(accuracy_score(bow_model.predict(X_test_bow), y_test))

assert accuracy_score(bow_model.predict(X_test_bow), y_test) > 0.7

NameError: name 'LogisticRegression' is not defined

In [ ]:
bow_model = LinearSVC(max_iter=70).fit(X_train_bow, y_train)
print(accuracy_score(bow_model.predict(X_test_bow), y_test))

assert accuracy_score(bow_model.predict(X_test_bow), y_test) > 0.68

### Модификация признаков (0.5 балла)

Добавьте title товара в bow с произвольным весом, как изменится качество?

In [23]:
def items_title_to_bow(items: np.array) -> np.array:
    """ Для каждого товара возвращает вектор его bow """
    # Давайте для начала попробуем строить bow только из description товара
    # assert ниже написан для bow из description
    matrix = [0]*len(items)
    for i in range(len(items)):
        matrix[i] = text_to_bow(items[i][0])
    return np.array(matrix)

X_train_title_bow = items_title_to_bow(X_train)
X_test_title_bow = items_title_to_bow(X_test)

New_train = X_train_title_bow + X_train_bow
New_test = X_test_title_bow + X_test_bow
bow_model = LogisticRegression(max_iter=100).fit(New_train, y_train)
print(accuracy_score(bow_model.predict(New_test), y_test))
bow_model = LinearSVC(max_iter=70).fit(New_train, y_train)
print(accuracy_score(bow_model.predict(New_test), y_test))

KeyboardInterrupt: 

Качество увеличилось значительно так как обучение проходит по большему колличеству слов описывающих товар 

Нормализуйте данные (`sklearn.preprocessing.normalize`) перед обучением. Что станет с качеством и почему?

In [ ]:
New_train = normalize(New_train)
New_test = normalize(New_test)
bow_model = LogisticRegression(max_iter=100).fit(New_train, y_train)
print(accuracy_score(bow_model.predict(New_test), y_test))
bow_model = LinearSVC(max_iter=70).fit(New_train, y_train)
print(accuracy_score(bow_model.predict(New_test), y_test))

Возможно, из-за того что мы исключаем масштабирование признаков, уменьшается отступ между разделяющей гиперплоскостью и объектами,
но в svc существуют вспомогательные гиперплоскости для решения проблем с этим отступом, поэтому svc показывает лучший accuracy, нежели лог регрессия при нормализации

### mystem (0.5) балла

Попробуйте обучиться, используя токенизатор mystem. Сравните качество.

In [15]:
!wget http://download.cdn.yandex.net/mystem/mystem-3.0-linux3.1-64bit.tar.gz
!tar -xvf mystem-3.0-linux3.1-64bit.tar.gz
!cp mystem /bin

"wget" ­Ґ пў«пҐвбп ў­гваҐ­­Ґ© Ё«Ё ў­Ґи­Ґ©
Є®¬ ­¤®©, ЁбЇ®«­пҐ¬®© Їа®Ја ¬¬®© Ё«Ё Ї ЄҐв­л¬ д ©«®¬.
tar: Error opening archive: Failed to open 'mystem-3.0-linux3.1-64bit.tar.gz'
cp: cannot stat 'mystem': No such file or directory


In [16]:
pip install git+https://github.com/nlpub/pymystem3

  Cloning https://github.com/nlpub/pymystem3 to c:\users\vaire\appdata\local\temp\pip-req-build-age7tmma
  Created wheel for pymystem3: filename=pymystem3-0.2.0-cp37-none-any.whl size=9926 sha256=b86477c8e623c51c9e02285b5bab459b471d4fa4cc6e512370693a35254285c5
  Stored in directory: C:\Users\vaire\AppData\Local\Temp\pip-ephem-wheel-cache-hweacptk\wheels\7d\75\c2\216a594291dee680749ce12c60d16125cfe1f363059e7163dc
Successfully built pymystem3


  Running command git clone -q https://github.com/nlpub/pymystem3 'C:\Users\vaire\AppData\Local\Temp\pip-req-build-age7tmma'


Note: you may need to restart the kernel to use updated packages.


In [17]:
from pymystem3 import Mystem

In [18]:
m = Mystem()

In [19]:
X_train_stem = copy.deepcopy(X_train)
X_test_stem = copy.deepcopy(X_test)

In [20]:
for i in tqdm(range(len(X_train))):
    for j in range(2):
        X_train_stem[i][j] = ''.join(m.lemmatize(X_train[i][j]))
for i in tqdm(range(len(X_test))):
    for j in range(2):
        X_test_stem[i][j] = ''.join(m.lemmatize(X_test[i][j]))

  0%|                                                                             | 6/21000 [01:35<81:36:56, 14.00s/it]

KeyboardInterrupt: 

In [ ]:
X_train_bow = items_to_bow(X_train_stem)
X_test_bow = items_to_bow(X_test_stem)
X_train_title_bow = items_title_to_bow(X_train_stem)
X_test_title_bow = items_title_to_bow(X_test_stem)

New_train_stem = X_train_title_bow + X_train_bow
New_test_stem = X_test_title_bow + X_test_bow

In [ ]:
bow_model = LogisticRegression(max_iter=100).fit(New_train_stem, y_train)
print(accuracy_score(bow_model.predict(New_test_stem), y_test))
bow_model = LinearSVC(max_iter=70).fit(New_train_stem, y_train)
print(accuracy_score(bow_model.predict(New_test_stem), y_test))

При использованни mystem Accuracy повышается, так как кроме обычной предобработке (lower) текстов мы на основе словаря обрабатываем слова и формировать морфологические гипотезы о незнакомых словах

### TF-IDF (1.5 балла)

Не все слова полезны одинаково, давайте попробуем [взвесить](http://tfidf.com/) их, чтобы отобрать более полезные.


> TF(t) = (Number of times term t appears in a document) / (Total number of terms in the document).
> 
> IDF(t) = log_e(Total number of documents / Number of documents with term t in it).


В sklearn есть TfidfVectorizer, но в этом задании его использовать нельзя.

In [ ]:
# Давайте для простоты считать один tf-idf для title и description.
# Для каждого слова из bow_vocabulary нужно посчитать
# в тексте скольких товаров встретилось это слово
new_train_list_tfidf = []
new_test_list_tfidf = []
for i in range(len(X_train)):
    k = preprocess(X_train[i][0]).split()
    k.extend((preprocess(X_train[i][1])).split())
    new_train_list_tfidf.append(k)
for i in range(len(X_test)):
    k = preprocess(X_test[i][0]).split()
    k.extend((preprocess(X_test[i][1])).split())
    new_test_list_tfidf.append(k)

In [ ]:
counter = 0
count_arr = {}
for i in tqdm(bow_vocabulary):
    for j in new_train_list_tfidf:
        if i in j:
            counter += 1
    count_arr.update({i: counter})
    counter = 0

In [ ]:
def text_to_tfidf(text: str, len_doc: int) -> np.array:
    """
    Возвращает вектор, где для каждого слова из most_common
    указан tf-idf
    """
    tf = np.zeros(10000)
    idf = np.zeros(10000)
    new_text = text.split()
    for i in new_text:
        if i in bow_vocabulary:
            tf[bow_vocabulary.index(i)] = new_text.count(i) / len(new_text)
            idf[bow_vocabulary.index(i)] = len_doc / count_arr[i]
 
    return np.array(tf*idf)

In [ ]:
tfidf_train = []
tfidf_test = []
for i in tqdm(range(len_train)):
    tfidf_train.append(text_to_tfidf(X_train_dt[i], len_train))
for i in tqdm(range(len_test)):
    tfidf_test.append(text_to_tfidf(X_test_dt[i], len_test))

In [ ]:
# Нормализуйте данные
tfidf_train = normalize(tfidf_train)
tfidf_test = normalize(tfidf_test)

### Модели на TF-IDF признаках (1 балл)

Обучите логистическую регрессию и SVC, оцените качество (accuracy_score)

In [ ]:
bow_model = LogisticRegression(max_iter=100).fit(tfidf_train, y_train)
print(accuracy_score(bow_model.predict(tfidf_test), y_test))
bow_model = LinearSVC(max_iter=70).fit(tfidf_train, y_train)
print(accuracy_score(bow_model.predict(tfidf_test), y_test))

Качество много меньше Mysteam и нормализованных данных с помощью BOW словаря. метод малоэффективен на этих данных.

### Hashing Vectorizer (0.5 балла)

Попробуйте использовать `sklearn.feature_extraction.text.HashingVectorizer` для векторизации текстов.
Обязательно оцените качество работы алгоритмов классификации с использованием новой векторизации.

In [ ]:
from sklearn.feature_extraction.text import HashingVectorizer
vectorizer = HashingVectorizer()

New_hashing_train = vectorizer.fit_transform(X_train_dt)
New_hashing_test = vectorizer.fit_transform(X_test_dt)
bow_model = LogisticRegression(max_iter=100).fit(New_hashing_train, y_train)
print(accuracy_score(bow_model.predict(New_hashing_test), y_test))
bow_model = LinearSVC(max_iter=70).fit(New_hashing_train, y_train)
print(accuracy_score(bow_model.predict(New_hashing_test), y_test))

Метод выдает потрясающий Accuracy на Avito-данных 8/10 

### Word Vectors (1 балл)

Давайте попробуем другой подход -- кажому слову сопоставим какой-то эмбеддинг (вектор).

Вектора будут небольшой размерности. Таким образом мы снизим количество параметров в модели.

Вектора мы возьмём уже готовые (обученные на текстах их интернета), так что наша модель будет знать некоторую дополнительную информацию о внешнем мире.

In [ ]:
!wget https://www.dropbox.com/s/0x7oxso6x93efzj/ru.tar.gz

In [ ]:
!tar -xzvf ru.tar.gz

In [ ]:
import gensim
from gensim.models.wrappers import FastText


model = FastText.load_fasttext_format('ru.bin')

In [ ]:
# Эмбеддинг предложения -- сумма эмбеддингов токенов


def sentence_embedding(sentence: str) -> np.array:
    """
    Складывает вектора токенов строки sentence
    """

    embedding_dim = model['кек'].shape[0]
    features = np.zeros([embedding_dim], dtype='float32')
    
    for word in sentence.split():
        if word in model:
            features += model[word]
    
    return features

In [ ]:
assert np.allclose(sentence_embedding('сдаётся уютный , тёплый гараж для стартапов в ml')[::50],
                   np.array([ 0.08189847,  0.07249198, -0.15601222,  0.03782297,  0.09215296, -0.23092946]))

In [ ]:
# Обучите логистическую регрессию и SVM
# Оцените качество (accuracy_score)
New_train_wv = [0]*len_train
New_test_wv = [0]*len_test

for i in range(len_train):
    New_train_wv[i] = sentence_embedding(X_train_dt[i])
for i in range(len_test):
    New_test_wv[i] = sentence_embedding(X_test_dt[i])

In [ ]:
wv_model = LogisticRegression(max_iter=100).fit(New_train_wv, y_train)
print(accuracy_score(wv_model.predict(New_test_wv), y_test))
wv_model = LinearSVC(max_iter=70).fit(New_train_wv, y_train)
print(accuracy_score(wv_model.predict(New_test_wv), y_test))

Accuracy LogisticRegression 0.578111111111  
Accuracy LinearSVC          0.499333333335

In [1]:
response = requests.get("https://pbs.twimg.com/media/EImSej_XkAAdBoB.jpg")
Image.open(BytesIO(response.content))

NameError: name 'requests' is not defined

### Что дальше?

Решение каждого пункта 1 балл:

1. N-Gram модели текстовой классификации

2. Обучиться на полных данных (контест на kaggle)

3. Поработать с другими эмбеддингами (word2vec, GloVe).

4. Использовать Vowpal Wabbit вместо sklearn.

5. Другие способы токенизации (pymorphy2, spaCy)


Снабжайте код пояснениями и графиками.
Обязательно необходимо написать вывод по каждому пункту, который вы реализуете.

# Другие способы токенизации

In [18]:
pip install pymorphy2

Note: you may need to restart the kernel to use updated packages.


In [35]:
from nltk.tokenize import RegexpTokenizer
tw = RegexpTokenizer(r'\w+')

xtrain = []
for lis in X_train: 
    lispr = (tw.tokenize(lis[0].lower()) + tw.tokenize(lis[1].lower()))
    xtrain.append(lispr)
    
xtest = []
for lis in X_test: 
    lispr = (tw.tokenize(lis[0].lower()) + tw.tokenize(lis[1].lower()))
    xtest.append(lispr)

In [30]:
m = MorphAnalyzer()

new_train = []
for line in tqdm(xtrain):
    sup_mas = []
    for token in range(len(line)):
        sup_mas.append(m.parse(line[token])[0].normal_form)
    new_train.append(' '.join(sup_mas))
    
new_test =  []
for line in tqdm(xtest):
    sup_mas = []
    for token in range(len(line)):
        sup_mas.append(m.parse(line[token])[0].normal_form)
    new_test.append(' '.join(sup_mas))

100%|██████████████████████████████████████████████████████████████████████████████| 9000/9000 [04:31<00:00, 33.10it/s]


In [33]:
bow_vocabulary = []
support_dict = {}
for i in new_train:
        for k in list(i.split()):
            try:
                support_dict[k] += 1
            except:
                support_dict.update({k:1})
sup_list = list(support_dict.items())
sup_list.sort(key=lambda x: x[1], reverse=True)
for i in range(10000):
    bow_vocabulary.append(sup_list[i][0])

In [34]:
X_train_bow = items_to_bow(new_train)
X_test_bow = items_to_bow(new_test)

100%|█████████████████████████████████████████████████████████████████████████████| 9000/9000 [00:26<00:00, 343.42it/s]


In [37]:
bow_model = LogisticRegression(max_iter=100).fit(X_train_bow, y_train)
print(accuracy_score(bow_model.predict(X_test_bow), y_test))
bow_model = LinearSVC(max_iter=70).fit(X_train_bow, y_train)
print(accuracy_score(bow_model.predict(X_test_bow), y_test))

C:\Users\vaire\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\vaire\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


0.2682222222222222
0.2723333333333333


Очень низкая accuracy возможно связанная с обновлением bow так как метод имеет видоизменения в морфологии слов.

# Обучение на всех данных

In [28]:
data = pd.read_csv("train.csv", index_col='id')

data.head()

ValueError: Index id invalid

In [ ]:
data.shape

In [ ]:
X = data[['title', 'description']].to_numpy()
y = data['Category'].to_numpy()

del data

In [ ]:
X_train2, y_train2 = X, y

In [ ]:
data = pd.read_csv("test.csv", index_col='id')

In [ ]:
data.shape

In [ ]:
X = data[['title', 'description']].to_numpy()
y = data['Category'].to_numpy()

del data

In [ ]:
X_test2, y_test2 = X, y

In [ ]:
len_train = len(X_train2)
len_test = len(X_test2)
X_train_dt = [0]*len_train
X_test_dt = [0]*len_test

In [ ]:
for i in range(len(X_train)):
    for j in range(len(X_train[i])):
        X_train[i][j] = preprocess(X_train[i][j])
for i in range(len(X_test)):
    for j in range(len(X_test[i])):
        X_test[i][j] = preprocess(X_test[i][j])

In [ ]:
bow_vocabulary = []
support_dict = {}
for i in X_train2:
    for j in i:
        for k in list(j.split()):
            try:
                support_dict[k] += 1
            except:
                support_dict.update({k:1})
sup_list = list(support_dict.items())
sup_list.sort(key=lambda x: x[1], reverse=True)
for i in range(10000):
    bow_vocabulary.append(sup_list[i][0])

In [ ]:
X_train_bow = items_to_bow(X_train2)
X_test_bow = items_to_bow(X_test2)
X_train_title_bow = items_title_to_bow(X_train2)
X_test_title_bow = items_title_to_bow(X_test2)

New_train = X_train_title_bow + X_train_bow
New_test = X_test_title_bow + X_test_bow
bow_model = LogisticRegression(max_iter=100).fit(New_train, y_train2)
print(accuracy_score(bow_model.predict(New_test), y_test2))
bow_model = LinearSVC(max_iter=70).fit(New_train, y_train2)
print(accuracy_score(bow_model.predict(New_test), y_test2))

Было бы хорошо если файл бы открылся  